# CH. 7 - TOPIC MODELS
## Activities

#### Activity 7.01

In [1]:
# not necessary
# added to suppress warnings coming from pyLDAvis

import warnings
warnings.filterwarnings('ignore')

In [2]:
import langdetect  # language detection
import matplotlib.pyplot  # plotting
import nltk  # natural language processing
import numpy  # arrays and matrices
import pandas  # dataframes
import pyLDAvis  # plotting
import pyLDAvis.sklearn  # plotting
import regex  # regular expressions
import sklearn  # machine learning

In [3]:
# define path

path = 'latimeshealth.txt'

In [4]:
# load data

df = pandas.read_csv(path, sep="|", header=None)
df.columns = ["id", "datetime", "tweettext"]

In [5]:
# define quick look function for data frame

def dataframe_quick_look(df, nrows):
    print("SHAPE:\n{shape}\n".format(shape=df.shape))
    print("COLUMN NAMES:\n{names}\n".format(names=df.columns))
    print("HEAD:\n{head}\n".format(head=df.head(nrows)))

In [6]:
dataframe_quick_look(df, nrows=2)

SHAPE:
(4171, 3)

COLUMN NAMES:
Index(['id', 'datetime', 'tweettext'], dtype='object')

HEAD:
                   id                        datetime  \
0  576760256031682561  Sat Mar 14 15:02:15 +0000 2015   
1  576715414811471872  Sat Mar 14 12:04:04 +0000 2015   

                                           tweettext  
0  Five new running shoes that aim to go the extr...  
1  Gym Rat: Disq class at Crunch is intense worko...  



In [7]:
# view final data that will be carried forward

raw = df['tweettext'].tolist()
print("HEADLINES:\n{lines}\n".format(lines=raw[:5]))
print("LENGTH:\n{length}\n".format(length=len(raw)))

HEADLINES:
['Five new running shoes that aim to go the extra mile http://lat.ms/1ELp3wU', 'Gym Rat: Disq class at Crunch is intense workout on pulley system http://lat.ms/1EKOFdr', 'Noshing through thousands of ideas at Natural Products Expo West http://lat.ms/1EHqywg', 'Natural Products Expo also explores beauty, supplements and more http://lat.ms/1EHqyfE', 'Free Fitness Weekends in South Bay beach cities aim to spark activity http://lat.ms/1EH3SMC']

LENGTH:
4171



In [8]:
# define function for checking language of tweets
# filter to english only

def do_language_identifying(txt):
    try:
       the_language = langdetect.detect(txt)
    except:
       the_language = 'none'
    return the_language

In [9]:
# define function to perform lemmatization

def do_lemmatizing(wrd):
    out = nltk.corpus.wordnet.morphy(wrd)
    return (wrd if out is None else out)

In [10]:
# define function to cleaning tweet data

def do_tweet_cleaning(txt):
    # identify language of tweet
    # return null if language not english
    lg = do_language_identifying(txt)
    if lg != 'en':
        return None
    
    # split the string on whitespace
    out = txt.split(' ')
    
    # identify screen names
    # replace with SCREENNAME
    out = ['SCREENNAME' if i.startswith('@') else i for i in out]
    
    # identify urls
    # replace with URL
    out = [
        'URL' if bool(regex.search('http[s]?://', i)) 
        else i for i in out
    ]
    
    # remove all punctuation
    out = [regex.sub('[^\\w\\s]|\n', '', i) for i in out]
    
    # make all non-keywords lowercase
    keys = ['SCREENNAME', 'URL']
    out = [i.lower() if i not in keys else i for i in out]
    
    # remove keywords
    out = [i for i in out if i not in keys]
    
    # remove stopwords
    list_stop_words = nltk.corpus.stopwords.words('english')
    list_stop_words = [regex.sub('[^\\w\\s]', '', i) for i in list_stop_words]
    
    out = [i for i in out if i not in list_stop_words]
    
    # lemmatizing
    out = [do_lemmatizing(i) for i in out]
    
    # keep words 4 or more characters long
    out = [i for i in out if len(i) >= 5]
    
    return out

In [11]:
# apply cleaning function to every tweet

clean = list(map(do_tweet_cleaning, raw))

In [12]:
# remove none types

clean = list(filter(None.__ne__, clean))
print("HEADLINES:\n{lines}\n".format(lines=clean[:5]))
print("LENGTH:\n{length}\n".format(length=len(clean)))

HEADLINES:
[['running', 'shoes', 'extra'], ['class', 'crunch', 'intense', 'workout', 'pulley', 'system'], ['thousand', 'natural', 'product'], ['natural', 'product', 'explore', 'beauty', 'supplement'], ['fitness', 'weekend', 'south', 'beach', 'spark', 'activity']]

LENGTH:
4095



In [13]:
# turn tokens back into strings
# concatenate using whitespaces

clean_sentences = [" ".join(i) for i in clean]

In [14]:
print(clean_sentences[0:10])

['running shoes extra', 'class crunch intense workout pulley system', 'thousand natural product', 'natural product explore beauty supplement', 'fitness weekend south beach spark activity', 'kayla harrison sacrifice', 'sonic treatment alzheimers disease', 'ultrasound brain restore memory alzheimers needle onlyso farin mouse', 'apple researchkit really medical research', 'warning chantix drink taking might remember']


#### Activity 7.02

In [15]:
# define global variables

number_words = 10
number_docs = 10
number_features = 1000

In [16]:
# bag of words conversion
# count vectorizer (raw counts)

vectorizer1 = sklearn.feature_extraction.text.CountVectorizer(
    analyzer="word",
    max_df=0.95, 
    min_df=10, 
    max_features=number_features
)
clean_vec1 = vectorizer1.fit_transform(clean_sentences)
print(clean_vec1[0])

feature_names_vec1 = vectorizer1.get_feature_names()

  (0, 320)	1


In [17]:
# define function to calculate perplexity based on number of topics

def perplexity_by_ntopic(data, ntopics):
    output_dict = {
        "Number Of Topics": [], 
        "Perplexity Score": []
    }
    
    for t in ntopics:
        lda = sklearn.decomposition.LatentDirichletAllocation(
            n_components=t,
            learning_method="online",
            random_state=0
        )
        lda.fit(data)
        
        output_dict["Number Of Topics"].append(t)
        output_dict["Perplexity Score"].append(lda.perplexity(data))
        
    output_df = pandas.DataFrame(output_dict)
    
    index_min_perplexity = output_df["Perplexity Score"].idxmin()
    output_num_topics = output_df.loc[
        index_min_perplexity,  # index
        "Number Of Topics"  # column
    ]
        
    return (output_df, output_num_topics)

In [18]:
# execute function on vector of numbers of topics
# takes several minutes

df_perplexity, optimal_num_topics = perplexity_by_ntopic(
    clean_vec1, 
    ntopics=[i for i in range(1, 21) if i % 2 == 0]
)

In [19]:
print(df_perplexity)

   Number Of Topics  Perplexity Score
0                 2        350.307315
1                 4        401.997523
2                 6        425.870398
3                 8        461.893848
4                10        475.041804
5                12        481.215539
6                14        505.080188
7                16        509.406520
8                18        527.267805
9                20        533.020490


In [20]:
# define and fit lda model

lda = sklearn.decomposition.LatentDirichletAllocation(
    n_components=optimal_num_topics,
    learning_method="online",
    random_state=0
)
lda.fit(clean_vec1)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [21]:
# define function to format raw output into nice tables

def get_topics(mod, vec, names, docs, ndocs, nwords):
    # word to topic matrix
    W = mod.components_
    W_norm = W / W.sum(axis=1)[:, numpy.newaxis]
    # topic to document matrix
    H = mod.transform(vec)
    
    W_dict = {}
    H_dict = {}
    
    for tpc_idx, tpc_val in enumerate(W_norm):
        topic = "Topic{}".format(tpc_idx)
        
        # formatting w
        W_indices = tpc_val.argsort()[::-1][:nwords]
        W_names_values = [
            (round(tpc_val[j], 4), names[j]) 
            for j in W_indices
        ]
        W_dict[topic] = W_names_values
        
        # formatting h
        H_indices = H[:, tpc_idx].argsort()[::-1][:ndocs]
        H_names_values = [
            (round(H[:, tpc_idx][j], 4), docs[j]) 
            for j in H_indices
        ]
        H_dict[topic] = H_names_values
        
    W_df = pandas.DataFrame(
        W_dict, 
        index=["Word" + str(i) for i in range(nwords)]
    )
    H_df = pandas.DataFrame(
        H_dict,
        index=["Doc" + str(i) for i in range(ndocs)]
    )
        
    return (W_df, H_df)

In [22]:
# get nice tables

W_df, H_df = get_topics(
    mod=lda,
    vec=clean_vec1,
    names=feature_names_vec1,
    docs=raw,
    ndocs=number_docs, 
    nwords=number_words
)

In [23]:
# word-topic table

print(W_df)

                    Topic0                Topic1
Word0      (0.0784, study)      (0.0414, latfit)
Word1     (0.0334, health)      (0.0308, cancer)
Word2      (0.024, people)     (0.0211, patient)
Word3      (0.0192, brain)       (0.0178, could)
Word4  (0.018, researcher)      (0.0168, doctor)
Word5      (0.0176, woman)       (0.0165, heart)
Word6     (0.0159, report)     (0.0147, disease)
Word7    (0.0154, obesity)  (0.0146, healthcare)
Word8  (0.0145, scientist)      (0.0145, weight)
Word9       (0.014, death)      (0.0105, expert)


In [24]:
# document-topic table

print(H_df)

                                                 Topic0  \
Doc0  (0.943, RT @LATgreatreads: Black babies are le...   
Doc1  (0.9373, Finally, here's some NON-anecdotal ev...   
Doc2  (0.9373, MT @DebbieGoffa ICYMI: Pioneering AID...   
Doc3  (0.9373, MT @LATerynbrown: Obamacare stories t...   
Doc4  (0.9373, This trial in monkeys is the stronges...   
Doc5  (0.9373, Side crunches can make love handles v...   
Doc6  (0.937, Once again, medical experts confirm th...   
Doc7  (0.9349, RT @LATerynbrown: "Finding the gene i...   
Doc8  (0.9346, Parents, give your kids advice about ...   
Doc9  (0.9333, RT @bfisher_qsi: #LATFit 17,500 steps...   

                                                 Topic1  
Doc0  (0.9438, RT @LATgreatreads: Many U.S. troops w...  
Doc1  (0.9406, Supplements to boost "low T" increase...  
Doc2  (0.938, “Creating a contest with somebody’s li...  
Doc3  (0.9374, Want to live for a long time? Prepare...  
Doc4  (0.9373, Scientists show how math can help hea...  
Do

In [26]:
# iteractive plot
# pca biplot and histogram

lda_plot = pyLDAvis.sklearn.prepare(lda, clean_vec1, vectorizer1, R=10)
pyLDAvis.display(lda_plot)

#### Activity 7.03

In [27]:
# bag of words conversion
# tf-idf method

vectorizer2 = sklearn.feature_extraction.text.TfidfVectorizer(
    analyzer="word",
    max_df=0.5, 
    min_df=20, 
    max_features=number_features,
    smooth_idf=False
)
clean_vec2 = vectorizer2.fit_transform(clean_sentences)
print(clean_vec2[0])

feature_names_vec2 = vectorizer2.get_feature_names()

In [28]:
# define and fit nmf model

nmf = sklearn.decomposition.NMF(
    n_components=optimal_num_topics,
    init="nndsvda",
    solver="mu",
    beta_loss="frobenius",
    random_state=0, 
    alpha=0.1, 
    l1_ratio=0.5
)
nmf.fit(clean_vec2)

NMF(alpha=0.1, beta_loss='frobenius', init='nndsvda', l1_ratio=0.5,
    max_iter=200, n_components=2, random_state=0, shuffle=False, solver='mu',
    tol=0.0001, verbose=0)

In [29]:
# get nicely formatted result tables

W_df, H_df = get_topics(
    mod=nmf,
    vec=clean_vec2,
    names=feature_names_vec2,
    docs=raw,
    ndocs=number_docs, 
    nwords=number_words
)

In [30]:
# word-topic table

print(W_df)

                  Topic0                Topic1
Word0    (0.3764, study)      (0.5933, latfit)
Word1   (0.0258, cancer)        (0.049, steps)
Word2   (0.0208, people)       (0.0449, today)
Word3  (0.0184, obesity)    (0.0405, exercise)
Word4   (0.0184, health)  (0.0274, healthtips)
Word5    (0.0181, brain)     (0.0258, workout)
Word6  (0.0173, suggest)     (0.0204, getting)
Word7   (0.0168, weight)     (0.0193, fitness)
Word8    (0.0153, woman)       (0.0143, great)
Word9    (0.0131, death)     (0.0132, morning)


In [31]:
# document-topic table

print(H_df)

                                                 Topic0  \
Doc0  (0.2032, How did H7N9 get from wild birds to c...   
Doc1  (0.2032, ICYMI MT @LATerynbrown @chadterhune L...   
Doc2  (0.2032, You can get kids to cut down on scree...   
Doc3  (0.2032, Avatars often reflect a reality about...   
Doc4  (0.2032, If you've been eating 5 servings of f...   
Doc5  (0.2032, Four economical workout machines will...   
Doc6  (0.2032, State Dept: Security at Libya consula...   
Doc7  (0.2032, California seeks limits on small-busi...   
Doc8  (0.2032, King Richard III's body provides some...   
Doc9  (0.2032, For seniors -- even seniors with memo...   

                                                 Topic1  
Doc0  (0.2276, Get too much sun today? An anti-infla...  
Doc1  (0.2276, Summer camp with a twist: Physical th...  
Doc2  (0.2276, How the month you were conceived can ...  
Doc3  (0.2276, Study: Contraceptives could prevent a...  
Doc4  (0.2276, At one point in the 1990s, 18% of hig...  
Do